# clean Zeno Files

# replace [\d]

In [2]:
import os

In [7]:
files = os.listdir("zeno_files/")

In [8]:
files[:2]

['Ahlefeld_Erna_zeno.txt', 'Ahlefeld_GesammelteErzählungen_zeno.txt']

In [10]:
texts = []

for file in files:
    with open("zeno_files/" + file, "r", encoding="utf8") as f:
        texts.append(f.read())

In [12]:
texts[0][:10]

'Ein liebes'

In [13]:
import re

In [21]:
cleaned_texts_with_ads = []

for text in texts:
    cleaned_texts_with_ads.append(re.sub("\[\d+\]", " ", text))

# scrape ads

In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
import time

In [24]:
df = pd.read_csv("metadata_Zeno.csv", encoding="utf8", sep = ";")

In [25]:
def get_html(website):
    try:
        with urllib.request.urlopen(website) as f:
           html_text = f.read()#.decode('utf-8')
    except urllib.error.URLError as e:
       print(e.reason)
    return html_text

In [28]:
def get_chapter_list(li_soup):
    chapters = []
    
    for li in li_soup:
            if li.b:
                continue
            else:
                if li.a:
                    for genre in list_of_genres:
                        if genre in li.a.get("href"):
                            chapters.append("http://www.zeno.org" + li.a.get("href"))
    return chapters

In [29]:
list_of_genres = ["/Roman/",
                  "/Romane/",
                  "/Erz%C3%A4hlungen/", # Erzählungen
                  "/M%C3%Archen/",
                  "/Erz%C3%Ahlprosa/",
                  "/Prosa/",
                  "/Epik/",
                  "/Legenden/",
                  "/Sagen/"] 

In [37]:
ad_texts = []

for text_website in list(df["sources"]):
    
    html_string = get_html(text_website)
    soup = BeautifulSoup(html_string, 'html.parser') 
    
    lis = soup.find_all("li",{'class': False, 'id': False})
    
    chapters = get_chapter_list(lis)
    
    if len(chapters)>0:
        for chapter in chapters:
            sub_html_string = get_html(chapter)
            sub_soup = BeautifulSoup(sub_html_string, 'html.parser')
            
            ads = soup.find_all("div", {"class":"holzingerAd"})

            for ad in ads:
                elements = ad.find_all("p")
                text = ""
                for element in elements:
                    text = text + element.text + "\n"
                ad_texts.append(text)
    else:
        ads = soup.find_all("div", {"class":"holzingerAd"})

        for ad in ads:
            text = ""
            elements = ad.find_all("p")
            for element in elements:
                text = text + element.text + "\n"
            ad_texts.append(text)
    
    
    time.sleep(1)

In [102]:
ads = list(set(ad_texts))

In [108]:
new_ad='Buchempfehlung\n\nBiedermeier - das klingt in heutigen Ohren nach langweiligem Spießertum, nach geschmacklosen rosa Teetässchen in Wohnzimmern, die aussehen wie Puppenstuben und in denen es irgendwie nach »Omma«  riecht.   Zu Recht. Aber nicht nur.   Biedermeier ist auch die Zeit einer zarten Literatur der Flucht ins Idyll, des Rückzuges ins private Glück und der Tugenden. Die Menschen im Europa nach Napoleon hatten die Nase voll von großen neuen Ideen, das aufstrebende Bürgertum forderte und entwickelte eine eigene Kunst und Kultur für sich, die unabhängig von feudaler Großmannssucht bestehen sollte.   Dass das gelungen ist, zeigt Michael Holzingers Auswahl von neun Meistererzählungen aus der sogenannten Biedermeierzeit.\n434 Seiten, 19.80 Euro\n'

ads.append(new_ad)

In [109]:
ads

['Buchempfehlung\n\nSchon der Titel, der auch damals kein geläufiges Synonym für »Autobiografie« war, zeigt den skurril humorvollen Stil des Autors Jean Paul, der in den letzten Jahren vor seiner Erblindung seine Jugenderinnerungen aufgeschrieben und in drei »Vorlesungen« angeordnet hat. »Ich bin ein Ich« stellt er dabei selbstbewußt fest.\n56 Seiten, 3.80 Euro\n',
 'Buchempfehlung\n\nZwei späte Novellen der Autorin, die feststellte: »Eine gescheite Frau hat Millionen geborener Feinde: alle dummen Männer.«\n72 Seiten, 4.80 Euro\n',
 'Buchempfehlung\n\nBiedermeier - das klingt in heutigen Ohren nach langweiligem Spießertum, nach geschmacklosen rosa Teetässchen in Wohnzimmern, die aussehen wie Puppenstuben und in denen es irgendwie nach »Omma«  riecht.   Zu Recht. Aber nicht nur.   Biedermeier ist auch die Zeit einer zarten Literatur der Flucht ins Idyll, des Rückzuges ins private Glück und der Tugenden. Die Menschen im Europa nach Napoleon hatten die Nase voll von großen neuen Ideen, da

In [110]:
len(ads)

21

In [111]:
cleaned_texts = []

for text in cleaned_texts_with_ads:
    for ad in ads:
        if re.search(ad, text):
            text = re.sub(ad, "", text)
    cleaned_texts.append(text)        
len(cleaned_texts)

503

In [88]:
len(texts)

503

# save files to corpus

In [113]:
for i in range(len(cleaned_texts)):
    with open("corpus_files/" + files[i], "w", encoding="utf8") as f:
        f.write(cleaned_texts[i])